### 핀테크 뉴스 수집

1. 핀테크 지원센터 홈페이지 → 알림마당 → 핀테크 정보 → 핀테크 뉴스에 접속
https://fintech.or.kr/web/board/boardContentsListPage.do?board_id=6&menu_id=6501&miv_pageNo=
2. 일자별로 있는 [11/18 핀테크 주요뉴스] 안에 있는 뉴스 제목과 원문 link 수집
3. 원문링크로 들어가서 뉴스 본문을 2025년 1월 2일까지 수집 후(약 18page) DB에 저장
4. 데이터 베이스에 저장된 뉴스를 데이터프레임으로 불러와서 텍스트 전처리, Wordcloud, LDA 토픽 모델링
5. 2025년 핀테크 뉴스에서 가장 많이 다룬 주제를 찾고 서비스 기획 아이디어 제시하기

In [11]:
import requests
import re
import os
import time
import pandas as pd
from bs4 import BeautifulSoup as bs

In [12]:
url="https://fintech.or.kr/web/board/boardContentsList.do"
payload = {"mode":"W", "board_id":"6","contents_id": "0579e2d50ca4449f97f5c21e6acf3d1c"}
r = requests.post(url, params=payload)
print(r.url)
print(r.status_code)
soup = bs(r.content, 'lxml')
soup

https://fintech.or.kr/web/board/boardContentsList.do?mode=W&board_id=6&contents_id=0579e2d50ca4449f97f5c21e6acf3d1c
200


<html><head><script language="javascript">
$(document).ready(function(){

	// 댓글수 노출여부 확인후 숨기기(미구현)
	
		$(".comment_cnt").hide();
	

	// 답변 노출여부 확인후 숨기기(미구현)
	
		$(".reply_status").hide();
	

	// 모바일 전용 페이지 숨기지
	//$(".mobile_list").hide();

	$(function() {

	});

	$("#searchtxt").keydown(function(key) {
		if (key.keyCode == 13) {
			search();
		}
	});
});
</script>
</head><body><div class="boardlist_top">
<ul class="box">
<li class="select">
<div class="optionbox">
<select id="searchkey" name="searchkey">
<option value="I">전체</option>
<option value="T">제목</option>
<option value="C">내용</option>
</select>
</div>
</li>
<li class="input">
<div class="inpbox"><input class="txt" id="searchtxt" name="searchtxt" placeholder="검색어 입력" title="검색어 입력" type="text" value=""/></div>
</li>
<li class="button">
<button class="btn3 bg_blue" onclick="search();" title="검색" type="button">검색</button>
<!-- <button type="button" class="btn3 bg_gray" title="재검색">재검색</button> -->
</li>
</ul>
</div>
<!--// list_t

In [15]:
list_url = "https://fintech.or.kr/web/board/boardContentsList.do"
board_id = "6"
page = 1

payload_list = {"mode": "W", "board_id": board_id, "pageIndex": page}
r_list = requests.post(list_url, data=payload_list)
soup_list = bs(r_list.content, "lxml")

rows = soup_list.select("table tbody tr")

for row in rows:
    date_td = row.select_one("td.last")
    date = date_td.get_text(strip=True) if date_td else "날짜없음"
    
    title_tag = row.select_one("td.txtl a")
    if title_tag:
        title = title_tag.get_text(strip=True)
        contents_id = title_tag["href"].split("'")[1]  # javascript:contentsView('ID')
    else:
        title = "제목없음"
        contents_id = None
    
    print(date, "-", title, "-", contents_id)

2025-11-12 - [경기도] 미래콘텐츠 성과공유회(AXR)에 초대합니다. (11/20(목) ~ 21(금)) - bf1b44ca61d74074a455cf21e180451c
2025-11-12 - [우리금융그룹] 2026 디노랩 서울 7기/부산 2기/경남 3기 모집(~12/03) - d4ad01f29e27456bacd3bf33c0668232
2025-09-17 - [금융보안원] 「데이터허브」 이용 안내 - b4b0b17c318d45158645c81c1472e29a
2024-02-20 - [국무조정실] 규제개혁신문고 제도 안내 - 51cb0e37c4064f2a985436e2ae3378df
2022-12-19 - [기술보증기금] 핀테크 기업을 위한 주요 보증상품 안내 - 3c24d05040984c79a7cdfd2c921952eb
2022-12-19 - [신용보증기금] 스타트업 지원 업무 안내 - 25f4360d5b9847a996828099e8d52ade
2022-06-20 - [한국무역보험공사] K-Sure 수출컨설팅 사업 안내 - 9c16609a99ad4c55914ffa9a7f2d58b7
2021-12-03 - [한국예탁결제원] 예탁결제원 보유 데이터 활용 안내 - 6b14460b5adc480aa050c4c535af8b72
2021-04-07 - [웰컴저축은행] 마이데이터 서비스 제휴 솔루션 - d4cbbeb3f9b3444cbf1856b48f9f2926
2025-11-20 - [11/20 핀테크 주요뉴스] - 12c003b7478b4c359c558ecc1edcc342
2025-11-19 - [11/19 핀테크 주요뉴스] - e5bc59a8918247a6943cc881219e5bec
2025-11-18 - [11/18 핀테크 주요뉴스] - 0579e2d50ca4449f97f5c21e6acf3d1c
2025-11-17 - [11/17 핀테크 주요뉴스] - bc4b472316354894a61a9d7515fd585a
2025-11-14 - [11/14 핀테크 주요뉴스

In [16]:
list_url = "https://fintech.or.kr/web/board/boardContentsList.do"
view_url = "https://fintech.or.kr/web/board/boardContentsView.do"

board_id = "6"
target_date = "2025-11-18"
max_pages = 18  
news_list = []

for page in range(1, max_pages + 1):
    payload_list = {"mode": "W", "board_id": board_id, "pageIndex": page}
    r_list = requests.post(list_url, data=payload_list, timeout=10)
    soup_list = bs(r_list.content, "lxml")
    
    rows = soup_list.select("table tbody tr")
    if not rows:
        break  
    
    for row in rows:
        date_td = row.select_one("td.last")
        if not date_td:
            continue
        date = date_td.get_text(strip=True)
        
        if date != target_date:
            continue  
        
        title_tag = row.select_one("td.txtl a")
        if not title_tag:
            continue
        title = title_tag.get_text(strip=True)
        contents_id = title_tag["href"].split("'")[1]  
        
        payload_view = {"board_id": board_id, "contents_id": contents_id}
        r_view = requests.get(view_url, params=payload_view, timeout=10)
        link = r_view.url
        
        news_list.append({"title": title, "link": link})

for news in news_list:
    print(news["title"], "-", news["link"])

[11/18 핀테크 주요뉴스] - https://fintech.or.kr/web/board/boardContentsView.do?board_id=6&contents_id=0579e2d50ca4449f97f5c21e6acf3d1c
[11/18 핀테크 주요뉴스] - https://fintech.or.kr/web/board/boardContentsView.do?board_id=6&contents_id=0579e2d50ca4449f97f5c21e6acf3d1c
[11/18 핀테크 주요뉴스] - https://fintech.or.kr/web/board/boardContentsView.do?board_id=6&contents_id=0579e2d50ca4449f97f5c21e6acf3d1c
[11/18 핀테크 주요뉴스] - https://fintech.or.kr/web/board/boardContentsView.do?board_id=6&contents_id=0579e2d50ca4449f97f5c21e6acf3d1c
[11/18 핀테크 주요뉴스] - https://fintech.or.kr/web/board/boardContentsView.do?board_id=6&contents_id=0579e2d50ca4449f97f5c21e6acf3d1c
[11/18 핀테크 주요뉴스] - https://fintech.or.kr/web/board/boardContentsView.do?board_id=6&contents_id=0579e2d50ca4449f97f5c21e6acf3d1c
[11/18 핀테크 주요뉴스] - https://fintech.or.kr/web/board/boardContentsView.do?board_id=6&contents_id=0579e2d50ca4449f97f5c21e6acf3d1c
[11/18 핀테크 주요뉴스] - https://fintech.or.kr/web/board/boardContentsView.do?board_id=6&contents_id=0579e2d50

In [13]:
list_url = "https://fintech.or.kr/web/board/boardContentsList.do"
view_url = "https://fintech.or.kr/web/board/boardContentsView.do"

target_date = "2025-11-18"  
board_id = "6"              
page = 1

news_list = []

while True:
    payload_list = {"mode": "W", "board_id": board_id, "pageIndex": page}
    r_list = requests.post(list_url, data=payload_list)
    soup_list = bs(r_list.content, "lxml")
    
    rows = soup_list.select("table tbody tr")
    if not rows:
        break  
    
    for row in rows:
        date_td = row.select_one("td.last")
        if not date_td:
            continue
        date = date_td.get_text(strip=True)
        
        if date != target_date:
            continue  # 원하는 날짜가 아니면 건너뜀
        
        title_tag = row.select_one("td.txtl a")
        if not title_tag:
            continue
        title = title_tag.get_text(strip=True)
        contents_id = title_tag["href"].split("'")[1]  # javascript:contentsView('ID')
        
        payload_view = {"board_id": board_id, "contents_id": contents_id}
        r_view = requests.get(view_url, params=payload_view)
        soup_view = bs(r_view.content, "lxml")

        link = r_view.url
        news_list.append({"title": title, "link": link})
    
    page += 1  

for news in news_list:
    print(news["title"], "-", news["link"])

KeyboardInterrupt: 